In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from plotly import graph_objects as go
import plotly.io as pio
import json
from plotly.offline import plot

In [2]:
data = pd.read_csv("./index_data.csv")
data['date'] = pd.to_datetime(data['date'])

In [3]:
data

,date,CSI300_Close,CSI1000_Close,LCVI_Close,SCGI_Close,WASI_Close,CSI300_PB,CSI1000_PB,LCVI_PB,SCGI_PB,WASI_PB
0,2009-12-01,3560.8310,4394.9900,4627.0835,4109.1788,2901.8079,3.2825,NaN,NaN,NaN,3.5214
1,2009-12-02,3597.3290,4474.9260,4666.2974,4176.4685,2934.1360,3.3154,NaN,NaN,NaN,3.5602
2,2009-12-03,3590.8760,4515.8350,4638.4644,4212.0841,2932.4329,3.3019,NaN,NaN,NaN,3.5557
3,2009-12-04,3643.4910,4375.9790,4789.9594,4108.7244,2950.3362,3.3776,NaN,NaN,NaN,3.5971
4,2009-12-07,3668.8320,4459.0790,4786.9781,4182.6524,2972.4902,3.3856,NaN,NaN,NaN,3.6160
...,...,...,...,...,...,...,...,...,...,...,...
3829,2025-09-03,4459.8296,7206.8758,8375.0882,5672.2713,6060.7115,1.4544,2.4242,0.9319,3.0013,1.7740
3830,2025-09-04,4365.2085,7041.1544,8374.7380,5494.0468,5938.1473,1.4385,2.3691,0.9366,2.9135,1.7488
3831,2025-09-05,4460.3249,7245.6670,8359.0742,5715.3562,6090.9215,1.4536,2.4347,0.9289,3.0382,1.7809
3832,2025-09-08,4467.5733,7311.0343,8359.2356,5794.1628,6133.7657,1.4548,2.4557,0.9289,3.0850,1.7903


#### **动量因子风格择时信号：**

##### 需要数据： 沪深300指数和中证1000指数日收盘价 
##### 信号逻辑： 取前第一日和第十一日指数收盘价计算过去十天指数收益率：
#####        (1) 沪深300指数收益率 > 中证1000指数收益率： 当天开盘买入沪深300指数,收盘平仓
#####        (2) 沪深300指数收益率 < 中证1000指数收益率： 当天开盘买入中证1000指数，收盘平仓
#####        (3) 若两者收益率均小于0：当天空仓

In [4]:

data["CSI300_Close_1"] = data['CSI300_Close'].shift()
data["CSI300_Close_11"] = data['CSI300_Close'].shift(11)
data["CSI1000_Close_1"] = data['CSI1000_Close'].shift()
data["CSI1000_Close_11"] = data['CSI1000_Close'].shift(11)
data['10_days_CSI300_rtn'] = data['CSI300_Close_1']/data['CSI300_Close_11']-1
data['10_days_CSI1000_rtn'] = data['CSI1000_Close_1']/data['CSI1000_Close_11']-1

In [5]:
data

,date,CSI300_Close,CSI1000_Close,LCVI_Close,SCGI_Close,WASI_Close,CSI300_PB,CSI1000_PB,LCVI_PB,SCGI_PB,WASI_PB,CSI300_Close_1,CSI300_Close_11,CSI1000_Close_1,CSI1000_Close_11,10_days_CSI300_rtn,10_days_CSI1000_rtn
0,2009-12-01,3560.8310,4394.9900,4627.0835,4109.1788,2901.8079,3.2825,NaN,NaN,NaN,3.5214,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-12-02,3597.3290,4474.9260,4666.2974,4176.4685,2934.1360,3.3154,NaN,NaN,NaN,3.5602,3560.8310,NaN,4394.9900,NaN,NaN,NaN
2,2009-12-03,3590.8760,4515.8350,4638.4644,4212.0841,2932.4329,3.3019,NaN,NaN,NaN,3.5557,3597.3290,NaN,4474.9260,NaN,NaN,NaN
3,2009-12-04,3643.4910,4375.9790,4789.9594,4108.7244,2950.3362,3.3776,NaN,NaN,NaN,3.5971,3590.8760,NaN,4515.8350,NaN,NaN,NaN
4,2009-12-07,3668.8320,4459.0790,4786.9781,4182.6524,2972.4902,3.3856,NaN,NaN,NaN,3.6160,3643.4910,NaN,4375.9790,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3829,2025-09-03,4459.8296,7206.8758,8375.0882,5672.2713,6060.7115,1.4544,2.4242,0.9319,3.0013,1.7740,4490.4528,4223.3741,7313.8824,7242.8494,0.063238,0.009807
3830,2025-09-04,4365.2085,7041.1544,8374.7380,5494.0468,5938.1473,1.4385,2.3691,0.9366,2.9135,1.7488,4459.8296,4271.3958,7206.8758,7305.4613,0.044115,-0.013495
3831,2025-09-05,4460.3249,7245.6670,8359.0742,5715.3562,6090.9215,1.4536,2.4347,0.9289,3.0382,1.7809,4365.2085,4288.0707,7041.1544,7253.3429,0.017989,-0.029254
3832,2025-09-08,4467.5733,7311.0343,8359.2356,5794.1628,6133.7657,1.4548,2.4557,0.9289,3.0850,1.7903,4460.3249,4377.9977,7245.6670,7362.9396,0.018805,-0.015927


In [6]:
def signal1_row(row):
    if (row['10_days_CSI300_rtn'] * row['10_days_CSI1000_rtn'] > 0) & (row['10_days_CSI300_rtn'] < 0):
        row['Signal1'] = 0
        row['Signal1_day_rtn'] = 1
        row['Position1'] = '空仓'
    elif row['10_days_CSI300_rtn'] > row['10_days_CSI1000_rtn']:
        row['Signal1'] = 1
        row['Signal1_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position1'] = '满仓沪深300指数'
    else: 
        row['Signal1'] = -1
        row['Signal1_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position1'] = '满仓中证1000指数'
    return row

data = data.apply(signal1_row, axis = 1)

In [7]:
# fig1 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal1_rtn'],
#                  text = data['Position1'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig1.update_layout(template='simple_white',
#                    title='<b>动量因子风格择时信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig1.show()

#### **拥挤度动量因子风格择时信号：**

##### 需要数据： 大盘价值指数PB,小盘成长指数PB，万得全A指数PB 
##### 信号逻辑： 取前第一日和第五日指数PB计算指标：
#####           R1 = (大盘价值指数PB-小盘成长指数PB)/万得全A指数PB
#####        (1) R1(t-1) > R1(t-5)： 当天开盘买入沪深300指数,收盘平仓
#####        (2) R1(t-1) < R1(t-5)： 当天开盘买入中证1000指数，收盘平仓

In [8]:
data['LCVI_PB_1'] = data['LCVI_PB'].shift()
data['LCVI_PB_5'] = data['LCVI_PB'].shift(5)
data['SCGI_PB_1'] = data['SCGI_PB'].shift()
data['SCGI_PB_5'] = data['SCGI_PB'].shift(5)
data['WASI_PB_1'] = data['WASI_PB'].shift()
data['WASI_PB_5'] = data['WASI_PB'].shift(5)

In [9]:
def signal2_row(row):
    if (row['LCVI_PB_1'] - row['SCGI_PB_1'])/row['WASI_PB_1'] > (row['LCVI_PB_5'] - row['SCGI_PB_5'])/row['WASI_PB_5']:
        row['Signal2'] = 1
        row['Signal2_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position2'] = '满仓沪深300指数'
    else: 
        row['Signal2'] = -1
        row['Signal2_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position2'] = '满仓中证1000指数'
    return row

data = data.apply(signal2_row, axis = 1)

In [10]:
# fig2 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal2_rtn'],
#                  text = data['Position2'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig2.update_layout(template='simple_white',
#                    title='<b>拥挤度动量因子风格择时信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081",
#                    ),
#                    width=2000, 
#                    height=500)
# fig2.show()

#### **相对强弱动量因子风格择时信号（大盘价值指数和小盘成长指数）**

##### 需要数据： 大盘价值指数、小盘成长指数、万得全A指数收盘价 
##### 信号逻辑： 取前第一日和第二日指数收盘价计算指标：
#####           d1 = 大盘价值指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####           d2 = 小盘成长指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####        (1) d1，d2小于0.2%： 空仓
#####        (2) d1 > d2: 择时沪深300指数
#####        (3) d1 < d2: 择时中证1000指数

In [11]:
data['LCVI_Close_1'] = data['LCVI_Close'].shift()
data['LCVI_Close_2'] = data['LCVI_Close'].shift(2)
data['SCGI_Close_1'] = data['SCGI_Close'].shift()
data['SCGI_Close_2'] = data['SCGI_Close'].shift(2)
data['WASI_Close_1'] = data['WASI_Close'].shift()
data['WASI_Close_2'] = data['WASI_Close'].shift(2)
data['LCVI_rtn_1'] = data['LCVI_Close_1']/data['LCVI_Close_2']
data['SCGI_rtn_1'] = data['SCGI_Close_1']/data['SCGI_Close_2']
data['WASI_rtn_1'] = data['WASI_Close_1']/data['WASI_Close_2']

In [12]:
def signal3_row(row):
    if (row['LCVI_rtn_1'] - row['WASI_rtn_1'] < 0.002) & (row['SCGI_rtn_1'] - row['WASI_rtn_1'] < 0.002):
        row['Signal3'] = 0
        row['Signal3_day_rtn'] = 1
        row['Position3'] = '空仓'
    elif row['LCVI_rtn_1'] > row['SCGI_rtn_1']:
        row['Signal3'] = 1
        row['Signal3_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position3'] = '满仓沪深300指数'
    else: 
        row['Signal3'] = -1
        row['Signal3_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position3'] = '满仓中证1000指数'
    return row

data = data.apply(signal3_row, axis = 1)

In [13]:
# fig3 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal3_rtn'],
#                  text = data['Position3'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig3.update_layout(template='simple_white',
#                    title='<b>相对强弱动量因子风格择时信号(大盘价值指数和小盘成长指数)<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig3.show()

#### **相对强弱动量因子风格择时信号（沪深300指数和中证1000指数）**

##### 需要数据： 沪深300指数、中证1000指数、万得全A指数收盘价 
##### 信号逻辑： 取前第一日和第二日指数收盘价计算指标：
#####           d1 = 沪深300指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####           d2 = 中证1000指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####        (1) d1，d2小于0.2%： 空仓
#####        (2) d1 > d2: 择时沪深300指数
#####        (3) d1 < d2: 择时中证1000指数

In [14]:
data['CSI300_Close_2'] = data['CSI300_Close'].shift(2)
data['CSI1000_Close_2'] = data['CSI1000_Close'].shift(2)
data['CSI300_rtn_1'] = data['CSI300_Close_1']/data['CSI300_Close_2']
data['CSI1000_rtn_1'] = data['CSI1000_Close_1']/data['CSI1000_Close_2']

In [15]:
def signal4_row(row):
    if (row['CSI300_rtn_1'] - row['WASI_rtn_1'] < 0.002) & (row['CSI1000_rtn_1'] - row['WASI_rtn_1'] < 0.002):
        row['Signal4'] = 0
        row['Signal4_day_rtn'] = 1
        row['Position4'] = '空仓'
    elif row['CSI300_rtn_1'] > row['CSI1000_rtn_1']:
        row['Signal4'] = 1
        row['Signal4_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position4'] = '满仓沪深300指数'
    else: 
        row['Signal4'] = -1
        row['Signal4_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position4'] = '满仓中证1000指数'
    return row

data = data.apply(signal4_row, axis = 1)

In [16]:
# fig4 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal4_rtn'],
#                  text = data['Position4'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig4.update_layout(template='simple_white',
#                    title='<b>相对强弱动量因子风格择时信号(沪深300指数和中证1000指数)<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig4.show()

#### **多维动量复合风格择时对冲信号**

##### 信号逻辑： 根据以上四个信号计算指标
#####          c1 = 择时沪深300指数信号数量
#####          c2 = 择时中证1000指数信号数量
#####         (1) c1 - c2 > 2: 7.5%多IF + 7.5%空IM
#####         (2) 0 < c1 - c2 < 2: 5%多IF + 5%空IM
#####         (3) c1 - c2 = 0: 空仓
#####         (4) -2 < c1 - c2 < 0: 5%多IM + 5%空IF
#####         (5) c1 - c2 < -2: 7.5%多IM + 7.5%空IF

In [17]:
margin = 0.12
data['Signal_all'] = data['Signal1'] + data['Signal2'] + data['Signal3'] + data['Signal4']

In [18]:
def signal5_row(row):
    if (row['Signal_all'] > 2):
        row['Signal_all_day_rtn'] = (row['CSI300_Close']/row['CSI300_Close_1']-1)*0.075/margin-(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.075/margin+1
        row['Position5'] = '7.5%多IF + 7.5%空IM'
    elif row['Signal_all'] > 0:
        row['Signal_all_day_rtn'] = (row['CSI300_Close']/row['CSI300_Close_1']-1)*0.05/margin-(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.05/margin+1
        row['Position5'] = '5%多IF + 5%空IM'
    elif row['Signal_all'] == 0:
        row['Signal_all_day_rtn'] = 1
        row['Position5'] = '空仓'
    elif row['Signal_all'] > -2:
        row['Signal_all_day_rtn'] = -(row['CSI300_Close']/row['CSI300_Close_1']-1)*0.05/margin+(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.05/margin+1
        row['Position5'] = '5%多IM + 5%空IF'
    else:
        row['Signal_all_day_rtn'] = -(row['CSI300_Close']/row['CSI300_Close_1']-1)*0.075/margin+(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.075/margin+1
        row['Position5'] = '7.5%多IM + 7.5%空IF'
    return row

data = data.apply(signal5_row, axis = 1)

In [19]:
# fig5 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal5_rtn'],
#                  text = data['Position5'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig5.update_layout(template='simple_white',
#                    title='<b>多维动量复合风格择时对冲信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig5.show()

In [20]:
data = data.loc[data['date'] > '2010-01-01',:].reset_index(drop=True)

In [21]:
data.iloc[-10:,[0,-1]].reset_index(drop = True)

,date,Position5
0,2025-08-27,5%多IM + 5%空IF
1,2025-08-28,空仓
2,2025-08-29,5%多IM + 5%空IF
3,2025-09-01,5%多IF + 5%空IM
4,2025-09-02,5%多IM + 5%空IF
5,2025-09-03,5%多IF + 5%空IM
6,2025-09-04,5%多IF + 5%空IM
7,2025-09-05,7.5%多IF + 7.5%空IM
8,2025-09-08,空仓
9,2025-09-09,7.5%多IM + 7.5%空IF


In [133]:
# data['Signal1_Value'] = np.cumprod(data['Signal1_day_rtn']) * 500000
# data['Signal2_Value'] = np.cumprod(data['Signal2_day_rtn']) * 500000
# data['Signal3_Value'] = np.cumprod(data['Signal3_day_rtn']) * 500000
# data['Signal4_Value'] = np.cumprod(data['Signal4_day_rtn']) * 500000
# data['Signal5_Value'] = np.cumprod(data['Signal_all_day_rtn']) * 500000